In [2]:
import requests
import configparser
import os

In [ ]:
config = configparser.ConfigParser()

# Ścieżka do pliku konfiguracyjnego
config_file_path = os.path.join(os.getcwd(), "config.ini")
config.read(config_file_path)

try:
    OPENROUTER_API_KEY = config['API']['OPENROUTER_API_KEY']
    PATH_SICK = config['Paths']['PATH_SICK']
    PATH_CONTROL = config['Paths']['PATH_CONTROL']
    MODEL_1 = config['Models']['MODEL_1']
    MODEL_2 = config['Models']['MODEL_2']
    print("Plik konfiguracyjny pomyślnie wczytany.")
except KeyError as e:
    print(f"Błąd: Brak klucza konfiguracyjnego: {e}\nUżywam ustawień domyślnych.")
    OPENROUTER_API_KEY = None
    # Ustawienia domyślne, dla jakich tworzono ten skrypt
    PATH_SICK = "data/descriptions_sick.xlsx"
    PATH_CONTROL = "data/descriptions_control_group.xlsx"
    MODEL_1 = "google/gemini-2.0-flash-exp:free"
    MODEL_2 = "deepseek/deepseek-chat-v3-0324:free"

Plik konfiguracyjny pomyślnie wczytany.


In [ ]:
# Wybór modelu do użycia
model = MODEL_1

if model == MODEL_1:
    input_file = "results/model1_full.txt"
else:
    input_file = "results/model2_full.txt"


# Wczytaj plik z lokalizacji
with open(input_file, "r", encoding="utf-8") as f:
    file_content = f.read()

#Odczytaj plik z opisami zmiennych
with open("data/disease_data.txt", "r", encoding="utf-8") as file:
    disease_data = file.read()

In [ ]:
# Prompt do modelu
prompt = f"""
Porównaj wyniki diagnozy HAE z załączonego pliku do poniższych informacji o chorobie.
Powiedz, jakie informacje są zawarte zarówno w pliku, jak i w informacjach poniżej.
Powiedz jakie są różnice: co występuje w pliku, a nie jest w informacjach poniżej, i na odwrót.
Zaznacz wyraźnie, jakie informacje są z jakiego źródła. Nie bierz pod uwagę innych źródeł.

INFORMACJE Z PLIKU:
{file_content}

INFORMACJE REFERENCYJNE:
{disease_data}
"""

In [ ]:
# Wysyłanie zapytania do OpenRouter
response = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers={
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    },
    json={
        "model": model,
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }
)

# Wyświetl odpowiedź
print(response.json()["choices"][0]["message"]["content"])

### **Porównanie informacji z pliku i informacji referencyjnych**  

#### **Wspólne elementy:**  
(Oba źródła zawierają następujące informacje):  

1. **Lokalizacja obrzęków:**  
   - Twarz (np. wargi, powieki), kończyny, narządy płciowe.  
   - Obrzęki narządów wewnętrznych (brzucha) i dróg oddechowych (krtań).  

2. **Objawy związane z obrzękiem jamy brzusznej:**  
   - Silne bóle brzucha, nudności, wymioty (potencjalnie prowadzące do hospitalizacji).  

3. **Charakter ataków:**  
   - Nagłe i nieprzewidywalne.  
   - Mogą trwać od kilku do kilkudziesięciu godzin/dni (plikt: 24–72 h; źródło referencyjne: 3–5 dni).  

4. **Czynniki wyzwalające:**  
   - Stres, infekcje, urazy fizyczne, zabiegi chirurgiczne/stomatologiczne.  

5. **Brak pokrzywki:**  
   - Obrzękom w HAE zwykle nie towarzyszy pokrzywka (w obu źródłach to cecha odróżniająca od alergii).  

6. **Ryzyko obrzęku krtani:**  
   - Oba źródła podają, że jest to stan zagrażający życiu i wymagający pilnej interwencji.  

---

#

In [ ]:
if model == MODEL_1:
    output_file = "comparison/porownanie_m1.txt"
else:
    output_file = "comparison/porownanie_m2.txt"

with open(output_file, "w", encoding="utf-8") as file:
    file.write(response.json()["choices"][0]["message"]["content"])